In [1]:
import tensorflow as tf, cv2, numpy as np, matplotlib.pyplot as plt, plotly.express as px
import os

Setting up gpu

In [2]:
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

Structure of archive is very (classes are located in separet directories and )

In [3]:
def image_getter(base_dir, skip_function):
    for part in os.listdir(base_dir):
        if skip_function(part):
            continue
        for img_class in os.listdir(os.path.join(base_dir,part)):
            for i in os.listdir(os.path.join(base_dir,part,img_class)):
                image = cv2.imread(os.path.join(base_dir,part,img_class,i),cv2.IMREAD_GRAYSCALE)
                image = np.reshape(image,image.shape+(1,))
                label=int(img_class.split('_')[0])-1

                yield image/255, label

def train_image_getter(base_dir="./../archive/leapGestRecog/", skip_dir='09'):
    return image_getter(base_dir, lambda x: skip_dir in x)
def test_image_getter(base_dir="./../archive/leapGestRecog/", leave_dir='09'):
    return image_getter(base_dir, lambda x: leave_dir not in x)

In [4]:
train = tf.data.Dataset.from_generator(train_image_getter, output_types=(tf.float32,tf.int8))
test = tf.data.Dataset.from_generator(test_image_getter, output_types=(tf.float32,tf.int8))
train_ds = train.shuffle(2000, reshuffle_each_iteration=True).repeat(5).batch(16)
test_ds = test.shuffle(2000, reshuffle_each_iteration=True).repeat(1).batch(16)
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False)

In [5]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=(240, 640,1)))
model.add(tf.keras.layers.MaxPooling2D((2, 4)))
model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 240, 640, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 120, 160, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 120, 160, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 80, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 80, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 40, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 40, 512)       1

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=.0002),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

In [8]:
tf.keras.backend.set_value(model.optimizer.learning_rate, 0.00001)

In [ ]:
history = model.fit(train_ds.as_numpy_iterator(), epochs=1, validation_data=test_ds.as_numpy_iterator())

In [ ]:
model.evaluate(a[0],a[1])

In [67]:
model.save("01_iteration.h5")